In [ ]:
#install the required libraries
!pip install pandas
!pip install ipython-sql prettytable 

import prettytable

prettytable.DEFAULT = 'DEFAULT'

In [4]:
#Load the pandas and sqlite3 libraries and establish a connection to FinalDB.db
import pandas as pd
import sqlite3

con = sqlite3.connect("FinalDB.db")
cur = con.cursor
%sql sqlite:///FinalDB.db

In [5]:
#Load the SQL magic module
%reload_ext sql

In [6]:
#Use Pandas to load the data

import csv
df = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCensusData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")

df2 = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoPublicSchools.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")

df3 = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DB0201EN-SkillsNetwork/labs/FinalModule_Coursera_V5/data/ChicagoCrimeData.csv?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDB0201ENSkillsNetwork20127838-2021-01-01")


In [7]:
#Establish a connection between SQL magic module and the database FinalDB.db
df.to_sql("CHICAGO_CENSUS_DATA", con, if_exists = 'replace', index = False, method = "multi")
df2.to_sql("CHICAGO_PUBLIC_SCHOOLS_DATA", con, if_exists = 'replace', index = False, method = "multi")
df3.to_sql("CHICAGO_CRIME_DATA", con, if_exists = 'replace', index = False, method = "multi")

533

In [ ]:
#Find the total number of crimes recorded in the CRIME table.

In [9]:
%%sql select count(*) from 
chicago_crime_data;

 * sqlite:///FinalDB.db
Done.


count(*)
533


In [ ]:
#List community area names and numbers with per capita income less than 11000

In [10]:
%%sql select COMMUNITY_AREA_NAME, COMMUNITY_AREA_NUMBER, PER_CAPITA_INCOME
    from CHICAGO_CENSUS_DATA
    where PER_CAPITA_INCOME < 11000;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,COMMUNITY_AREA_NUMBER,PER_CAPITA_INCOME
West Garfield Park,26.0,10934
South Lawndale,30.0,10402
Fuller Park,37.0,10432
Riverdale,54.0,8201


In [ ]:
#List all case numbers for crimes involving minors?(children are not considered minors for the purposes of crime analysis)

In [11]:
%%sql SELECT CASE_NUMBER
FROM chicago_crime_data
WHERE DESCRIPTION LIKE '%MINOR%'
  AND DESCRIPTION NOT LIKE '%CHILD%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER
HL266884
HK238408


In [ ]:
#List all kidnapping crimes involving a child?

In [12]:
%%sql SELECT CASE_NUMBER,PRIMARY_TYPE, DESCRIPTION
FROM chicago_crime_data
WHERE PRIMARY_TYPE = 'KIDNAPPING'
  AND DESCRIPTION LIKE '%CHILD%';

 * sqlite:///FinalDB.db
Done.


CASE_NUMBER,PRIMARY_TYPE,DESCRIPTION
HN144152,KIDNAPPING,CHILD ABDUCTION/STRANGER


In [ ]:
#List the kind of crimes that were recorded at schools. (No repetitions)

In [13]:
%%sql SELECT DISTINCT PRIMARY_TYPE
FROM chicago_crime_data
WHERE LOCATION_DESCRIPTION LIKE '%SCHOOL%';

 * sqlite:///FinalDB.db
Done.


PRIMARY_TYPE
BATTERY
CRIMINAL DAMAGE
NARCOTICS
ASSAULT
CRIMINAL TRESPASS
PUBLIC PEACE VIOLATION


In [ ]:
#List the type of schools along with the average safety score for each type.

In [14]:
%%sql SELECT 
    "Elementary, Middle, or High School" AS School_Type,
    AVG(SAFETY_SCORE) AS Average_Safety_Score
FROM 
    chicago_public_schools_data
WHERE 
    SAFETY_SCORE IS NOT NULL
GROUP BY 
    "Elementary, Middle, or High School";

 * sqlite:///FinalDB.db
Done.


School_Type,Average_Safety_Score
ES,49.52038369304557
HS,49.62352941176471
MS,48.0


In [ ]:
#List 5 community areas with highest % of households below poverty line

In [15]:
%%sql SELECT COMMUNITY_AREA_NAME, PERCENT_HOUSEHOLDS_BELOW_POVERTY
FROM chicago_census_data
ORDER BY PERCENT_HOUSEHOLDS_BELOW_POVERTY DESC
LIMIT 5;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME,PERCENT_HOUSEHOLDS_BELOW_POVERTY
Riverdale,56.5
Fuller Park,51.2
Englewood,46.6
North Lawndale,43.1
East Garfield Park,42.4


In [ ]:
#Which community area is most crime prone? Display the coumminty area number only

In [16]:
%%sql SELECT COMMUNITY_AREA_NUMBER
FROM chicago_crime_data
GROUP BY COMMUNITY_AREA_NUMBER
ORDER BY COUNT(*) DESC
LIMIT 1;

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NUMBER
25.0


In [ ]:
#Use a sub-query to find the name of the community area with highest hardship index

In [17]:
%%sql SELECT COMMUNITY_AREA_NAME
FROM chicago_census_data
WHERE HARDSHIP_INDEX = (
    SELECT MAX(HARDSHIP_INDEX)
    FROM chicago_census_data
);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Riverdale


In [ ]:
#Use a sub-query to determine the Community Area Name with most number of crimes?

In [18]:
%%sql SELECT COMMUNITY_AREA_NAME
FROM chicago_census_data
WHERE COMMUNITY_AREA_NUMBER = (
    SELECT COMMUNITY_AREA_NUMBER
    FROM chicago_crime_data
    GROUP BY COMMUNITY_AREA_NUMBER
    ORDER BY COUNT(*) DESC
    LIMIT 1
);

 * sqlite:///FinalDB.db
Done.


COMMUNITY_AREA_NAME
Austin
